<a href="https://colab.research.google.com/drive/1ZH-w_xLJV4v-oCd-GtfYA9FWaloJVMqv" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**CentAI**
By Justin Kim

# DESCRIPTION
This is an AI project to play a game.

It uses Deep Reinforcement Learning (DQNAgent)

https://keon.io/deep-q-learning/

The game is Centakill, made by one of my friends, Kaiway Tang.

Go check them out at: https://kaiway.itch.io/

Or just the game at: https://kaiway.itch.io/centa



NOTE:

This only works if you run this on your local machine.

You can do this by:

1. copying the code into a local Python environment on your computer.
2. connecting this notebook to a local runtime
https://research.google.com/colaboratory/local-runtimes.html

I would recommend option 2.


For Option 1:

I will assume you have basic Python knowledge and know how to use your Python editor (installing packages, creating new files) for this tutorial.

1. Install the packages listed below
2. Create a new file
3. Copy code from each code box into the file
4. Run the program
5. If prompted, give the editor access to your system (so it can use press keys)
6. If there are any questions, add a comment to my Youtube video and I'll try to help


For Option 2:

It should be simpler, just downloading a couple things and enabling them, so I will only assume you know how to use your computer.

1. Download Python https://www.python.org/downloads/

2. Go to your terminal

3. Do ```pip install jupyterlab```

4. Do ```pip install jupyter_https_over_ws```

5. Do ```jupyter serverextension enable --py jupyter_https_over_ws```

6. Do 
```
jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com'  --port=8888 --NotebookApp.port_retries=0
```
to start the notebook server. This should open a new tab in your web browser.

7. Go back to this colab to the top right where it says RAM and Disk and click on the dropdown menu

8. Press connect to local runtime and then press connect

#Code

installaton of various packages

run only once at the beginning


In [0]:
!pip install numpy
!pip install keras
!pip install mss
!pip install Pillow
!pip install opencv-python
!pip install pyautogui
!pip install pytesseract

imports:

In [0]:
from collections import deque
import random
import numpy as np
import time

from keras.models import Sequential, model_from_json
from keras.layers import Dense
from keras.optimizers import Adam

from mss import mss
from PIL import Image
import cv2

import pyautogui
import pytesseract

This is DQNAgent. It was taken from https://keon.io/deep-q-learning/

This is a class that contains the model.

Tweak the values until the model learns better, these are just the default values.



In [0]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=50000)
        self.gamma = 0.95  # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, min(batch_size, len(self.memory)))
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * \
                         np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def save(self):
        self.model.save_weights('model.h5')

    def load(self):
        self.model.load_weights('model.h5')


Functions for the model to get input and interact with the computer.

Run this cell once.

In [0]:
def capture_ss():
    img = sc.grab(monitor=sc.monitors[1])
    return np.array(Image.frombytes('RGB', img.size, img.bgra, 'raw', 'BGRX'))


def process_img(original_img):
    processed_img = cv2.resize(original_img, scr_dim, interpolation=cv2.INTER_AREA)
    processed_img = cv2.cvtColor(processed_img, cv2.COLOR_BGR2GRAY)
    return processed_img


def shape_img(processed_img):
    shaped_img = np.reshape(processed_img, [1, screen_size])
    return shaped_img


def act(action):
    print(action)
    if action > 4:
        if action < 20:
            if action % 4 == 0:
                pyautogui.keyDown('W')
            if action % 4 == 1:
                pyautogui.keyDown('A')
            if action % 4 == 2:
                pyautogui.keyDown('S')
            if action % 4 == 3:
                pyautogui.keyDown('D')
        else:
            if action == 20 or action == 21 or action == 24 or action == 25 or action == 28 or action == 29:
                pyautogui.keyDown('W')
            if action == 20 or action == 22 or action == 24 or action == 26 or action == 28 or action == 30:
                pyautogui.keyDown('A')
            if action == 23 or action == 22 or action == 27 or action == 26 or action == 31 or action == 30:
                pyautogui.keyDown('S')
            if action == 23 or action == 21 or action == 27 or action == 25 or action == 31 or action == 29:
                pyautogui.keyDown('D')
    if action == 0 or (8 <= action <= 11) or (20 <= action <= 23):
        pyautogui.keyDown('U')
    if action == 1 or (12 <= action <= 15) or (24 <= action <= 27):
        pyautogui.keyDown('I')
    if action == 2:
        pyautogui.keyDown('O')
    if action == 3 or (16 <= action <= 19) or (28 <= action <= 31):
        pyautogui.keyDown('P')


def reset():
    pyautogui.keyUp('W')
    pyautogui.keyUp('A')
    pyautogui.keyUp('S')
    pyautogui.keyUp('D')
    pyautogui.keyUp('U')
    pyautogui.keyUp('I')
    pyautogui.keyUp('O')
    pyautogui.keyUp('P')


The main function where all the AI happens.

Run this once.

In [0]:
def main():
    agent = DQNAgent(screen_size, 4 + 4 + 12 + 12)
    # agent.load()

    death_flag = process_img(capture_ss())[death_loc[1], death_loc[0]]
    print("starting")

    time.sleep(2)

    # repeat episode_num times
    for e in range(episode_num):
        pyautogui.keyDown('tab')
        pyautogui.keyUp('tab')

        score = 0
        dead = False
        last_scr = process_img(capture_ss())
        last_scr_vector = shape_img(last_scr)
        action = agent.act(last_scr_vector)
        act(action)
        last_time = time.time()

        while not dead:
            reset()
            screen = capture_ss()
            pro_scr = process_img(screen)
            scr_vector = shape_img(pro_scr)

            score_crop = screen[score_loc[1]:score_loc[1] + score_dim[1],
                         score_loc[0]:score_loc[0] + score_dim[0]]
            score_ocr = pytesseract.image_to_string(score_crop,
                                                    config='--psm 10 --oem 3 -c tessedit_char_whitelist=0123456789')
            if score_ocr != '':
                score = max(score, int(score_ocr))

            death_check = pro_scr[death_loc[1]][death_loc[0]]
            if death_flag == death_check:
                dead = True

            reward = score

            agent.remember(last_scr_vector, action, reward, scr_vector, dead)

            if dead:
                print('episode: {}/{}, score: {}'.format(e, episode_num, reward))
                continue

            print(time.time() - last_time)
            last_time = time.time()
            action = agent.act(scr_vector)
            act(action)
            last_scr = pro_scr
            last_scr_vector = scr_vector

        agent.replay(32)

    agent.save()


This cell runs the main function. Run as many times as you want.

You may need to change score_loc and score_dim to fit your computer size. 

In [0]:
episode_num = 10
scr_dim = (200, 50)
score_loc = (1085, 150)
score_dim = (400, 150)
death_loc = (int(scr_dim[0] / 2), int(scr_dim[1] / 2))
screen_size = scr_dim[0] * scr_dim[1]

sc = mss()

main()
